# Movie Genre NLP

## Imports 

In [310]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import json
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional
import os

## Loading DataSet


In [311]:
df = pd.read_csv("/Users/chaitanyasingh/Documents/CollegeStuff/Project/1/n_movies.csv")
df.head()

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413"


In [312]:
def clean_text(text):
    return re.sub(r'\W+', ' ', text).lower().strip()

In [313]:
# Cleaning dataframe 
data = df # duplicate dataframe
df.drop(['title', 'year', 'certificate', 'duration', 'rating', 'stars', 'votes'], axis = 1)

df['clean_description'] = df['description'].apply(clean_text)

data_filtered = data.dropna(subset=['genre'])
data_filtered['genre_list'] = data_filtered['genre'].apply(lambda x: x.split(', '))

In [314]:
mlb = MultiLabelBinarizer()
encoded_genres = mlb.fit_transform(data_filtered['genre_list'])

In [315]:
X_train, X_test, y_train, y_test = train_test_split(data_filtered['clean_description'], encoded_genres, test_size=0.3, random_state=42)

In [316]:
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [317]:
max_length = max(len(x) for x in X_train_seq)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

In [318]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=100, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    GlobalAveragePooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(mlb.classes_), activation='sigmoid')
])
tokenizer_json = tokenizer.to_json()
with open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))
joblib.dump(mlb, 'mlb.pkl')

['mlb.pkl']

In [319]:
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [320]:
%%time
model.fit(X_train_pad, y_train, epochs=55, validation_data=(X_test_pad, y_test))

Epoch 1/55
217/217 [==============================] - 9s 34ms/step - loss: 0.2974 - accuracy: 0.1528 - val_loss: 0.2327 - val_accuracy: 0.1790
Epoch 2/55
217/217 [==============================] - 7s 31ms/step - loss: 0.2393 - accuracy: 0.2080 - val_loss: 0.2250 - val_accuracy: 0.2657
Epoch 3/55
217/217 [==============================] - 7s 32ms/step - loss: 0.2269 - accuracy: 0.2729 - val_loss: 0.2194 - val_accuracy: 0.2977
Epoch 4/55
217/217 [==============================] - 7s 31ms/step - loss: 0.2171 - accuracy: 0.3242 - val_loss: 0.2188 - val_accuracy: 0.2997
Epoch 5/55
217/217 [==============================] - 7s 33ms/step - loss: 0.2082 - accuracy: 0.3393 - val_loss: 0.2152 - val_accuracy: 0.2984
Epoch 6/55
217/217 [==============================] - 7s 32ms/step - loss: 0.1991 - accuracy: 0.3326 - val_loss: 0.2102 - val_accuracy: 0.2771
Epoch 7/55
217/217 [==============================] - 7s 32ms/step - loss: 0.1891 - accuracy: 0.3481 - val_loss: 0.2064 - val_accuracy: 0.3149

In [321]:
model.save('movie_genre_classifier.h5')